In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# pandas の表示オプションを全表示に設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# データフォルダのパス
data_folder = 'edit_data'

# データフォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

df = pd.DataFrame()

# 各CSVファイルを処理
for file_name in csv_files:
    file_path = os.path.join(data_folder, file_name)
    temp_df = pd.read_csv(file_path)
    df = pd.concat([df, temp_df], ignore_index=True)
    
# 参加者の数を確認
num_participants = df['participant_id'].nunique()
print(f'被験者数: {num_participants}')

被験者数: 26


In [3]:
df_F = df[(df['TorF'] == 'F') & (df['input_text'].notna()) & (df['angle'] != 0)].copy()

# scaleの順序を指定してカテゴリ化
scale_order = ['intended', '50cm', '75cm', '100cm']
df_F['scale'] = pd.Categorical(df_F['scale'], categories=scale_order, ordered=True)

# angle, period を数値に変換（必要に応じて）
df_F['angle'] = pd.to_numeric(df_F['angle'], errors='coerce')
df_F['period'] = pd.to_numeric(df_F['period'], errors='coerce')
df_F['line_width'] = pd.to_numeric(df_F['line_width'], errors='coerce')

# 指定順（scale -> angle昇順 -> period昇順）でソート
df_F = df_F.sort_values(by=['scale', 'angle', 'period', 'line_width']).reset_index(drop=True)

# input_text を整数表示に変換（欠損値がある場合でも表示できるよう nullable Int64 を使用）
df_F['input_text'] = pd.to_numeric(df_F['input_text'], errors='coerce')
if df_F['input_text'].isna().any():
    df_F['input_text'] = df_F['input_text'].astype('Int64')  # pandas nullable integer dtype
else:
    df_F['input_text'] = df_F['input_text'].astype(int)

In [4]:
import difflib

# 比較対象の列名
reference_col = 'text' 

def levenshtein(a: str, b: str) -> int:
    a = a or ""
    b = b or ""
    n, m = len(a), len(b)
    if n == 0:
        return m
    if m == 0:
        return n
    dp = list(range(m + 1))
    for i in range(1, n + 1):
        prev, dp[0] = dp[0], i
        for j in range(1, m + 1):
            cur = dp[j]
            if a[i - 1] == b[j - 1]:
                dp[j] = prev
            else:
                dp[j] = 1 + min(prev, dp[j], dp[j - 1])
            prev = cur
    return dp[m]

def label_miss(ref, hyp):
    if pd.isna(ref) or pd.isna(hyp):
        return None
    ref = str(ref)
    hyp = str(hyp)
    sm = difflib.SequenceMatcher(None, ref, hyp)
    ops = sm.get_opcodes()
    parts = []
    for tag, i1, i2, j1, j2 in ops:
        if tag == 'equal':
            continue
        ref_seg = ref[i1:i2]
        hyp_seg = hyp[j1:j2]
        if tag == 'replace':
            parts.append(f"replace: '{ref_seg}' -> '{hyp_seg}'")
        elif tag == 'delete':
            parts.append(f"delete: '{ref_seg}'")
        else:  # insert
            parts.append(f"insert: '{hyp_seg}' ")
    dist = levenshtein(ref, hyp)
    summary = f"LD={dist}"
    if parts:
        summary += "; " + "; ".join(parts)
    return summary

# df に miss 列を追加
if reference_col in df_F.columns:
    df_F['miss'] = df_F.apply(lambda r: label_miss(r[reference_col], r.get('input_text')), axis=1)
else:
    # reference_col がない場合は通知用の文字列を入れる（必要なら先に列を作成してください）
    df_F['miss'] = None

display(df_F[['participant_id', 'scale', 'angle', 'period', 'line_width','text', 'input_text','miss']])

,participant_id,scale,angle,period,line_width,text,input_text,miss
0,402,intended,45.0,7.0,3.0,3953,3353,LD=1; replace: '9' -> '3'
1,406,intended,45.0,7.0,3.0,7493,7433,LD=1; delete: '9'; insert: '3'
2,408,intended,45.0,7.0,3.0,4874,7874,LD=1; replace: '4' -> '7'
3,302,intended,45.0,8.0,2.0,6884,5884,LD=1; replace: '6' -> '5'
4,308,intended,45.0,8.0,2.0,8393,8397,LD=1; replace: '3' -> '7'
5,416,intended,45.0,8.0,2.0,7533,7553,LD=1; insert: '5' ; delete: '3'
6,302,intended,45.0,8.0,3.0,6856,5856,LD=1; replace: '6' -> '5'
7,306,intended,45.0,8.0,3.0,2033,2032,LD=1; replace: '3' -> '2'
8,416,intended,90.0,3.0,2.0,4672,2672,LD=1; replace: '4' -> '2'
9,411,intended,90.0,4.0,2.0,5315,5313,LD=1; replace: '5' -> '3'


In [5]:
# angle 別に miss をカウント（NaN を含む）
angle_miss = df_F.groupby('angle')['miss'].value_counts(dropna=False).rename('count').reset_index()
angle_miss = angle_miss.sort_values(['angle', 'count'], ascending=[True, False]).reset_index(drop=True)
display(angle_miss)

# 見やすい表形式でも表示（pivot）
angle_miss_table = df_F.groupby('angle')['miss'].value_counts(dropna=False).unstack(fill_value=0).reindex().reset_index()
display(angle_miss_table)

,angle,miss,count
0,45.0,LD=1; replace: '5' -> '6',7
1,45.0,LD=1; replace: '3' -> '2',6
2,45.0,LD=1; replace: '9' -> '5',6
3,45.0,LD=1; replace: '0' -> '6',5
4,45.0,LD=1; replace: '0' -> '8',5
5,45.0,LD=1; replace: '8' -> '9',5
6,45.0,LD=1; replace: '6' -> '5',4
7,45.0,LD=1; replace: '8' -> '5',4
8,45.0,LD=1; replace: '0' -> '9',3
9,45.0,LD=1; replace: '2' -> '7',3


miss,angle,LD=1; delete: '0'; insert: '3',LD=1; delete: '0'; insert: '8',LD=1; delete: '0'; insert: '9',LD=1; delete: '1',LD=1; delete: '1'; insert: '4',LD=1; delete: '2'; insert: '3',LD=1; delete: '2'; insert: '9',LD=1; delete: '3'; insert: '5',LD=1; delete: '3'; insert: '9',LD=1; delete: '5'; insert: '3',LD=1; delete: '5'; insert: '6',LD=1; delete: '5'; insert: '8',LD=1; delete: '5'; insert: '9',LD=1; delete: '6'; insert: '3',LD=1; delete: '7'; insert: '2',LD=1; delete: '8'; insert: '0',LD=1; delete: '8'; insert: '3',LD=1; delete: '8'; insert: '6',LD=1; delete: '9'; insert: '2',LD=1; delete: '9'; insert: '3',LD=1; delete: '9'; insert: '5',LD=1; delete: '95'; insert: '99',LD=1; insert: '0' ; delete: '2',LD=1; insert: '0' ; delete: '9',LD=1; insert: '1' ; delete: '2',LD=1; insert: '1' ; delete: '3',LD=1; insert: '15' ; delete: '12',LD=1; insert: '2' ; delete: '3',LD=1; insert: '3' ; delete: '1',LD=1; insert: '3' ; delete: '2',LD=1; insert: '5',LD=1; insert: '5' ; delete: '3',LD=1; insert: '5' ; delete: '6',LD=1; insert: '6' ; delete: '2',LD=1; insert: '6' ; delete: '5',LD=1; insert: '6' ; delete: '8',LD=1; insert: '7' ; delete: '3',LD=1; insert: '75' ; delete: '78',LD=1; insert: '8' ; delete: '3',LD=1; insert: '9' ; delete: '6',LD=1; insert: '9' ; delete: '8',LD=1; replace: '0' -> '1',LD=1; replace: '0' -> '2',LD=1; replace: '0' -> '3',LD=1; replace: '0' -> '5',LD=1; replace: '0' -> '6',LD=1; replace: '0' -> '7',LD=1; replace: '0' -> '8',LD=1; replace: '0' -> '9',LD=1; replace: '1' -> '0',LD=1; replace: '1' -> '2',LD=1; replace: '1' -> '3',LD=1; replace: '1' -> '4',LD=1; replace: '1' -> '5',LD=1; replace: '1' -> '7',LD=1; replace: '2' -> '0',LD=1; replace: '2' -> '3',LD=1; replace: '2' -> '5',LD=1; replace: '2' -> '6',LD=1; replace: '2' -> '7',LD=1; replace: '2' -> '8',LD=1; replace: '2' -> '9',LD=1; replace: '3' -> '0',LD=1; replace: '3' -> '1',LD=1; replace: '3' -> '2',LD=1; replace: '3' -> '4',LD=1; replace: '3' -> '5',LD=1; replace: '3' -> '6',LD=1; replace: '3' -> '7',LD=1; replace: '3' -> '8',LD=1; replace: '3' -> '9',LD=1; replace: '4' -> '0',LD=1; replace: '4' -> '1',LD=1; replace: '4' -> '2',LD=1; replace: '4' -> '5',LD=1; replace: '4' -> '6',LD=1; replace: '4' -> '7',LD=1; replace: '4' -> '8',LD=1; replace: '4' -> '9',LD=1; replace: '5' -> '0',LD=1; replace: '5' -> '2',LD=1; replace: '5' -> '3',LD=1; replace: '5' -> '4',LD=1; replace: '5' -> '6',LD=1; replace: '5' -> '8',LD=1; replace: '5' -> '9',LD=1; replace: '6' -> '0',LD=1; replace: '6' -> '2',LD=1; replace: '6' -> '3',LD=1; replace: '6' -> '4',LD=1; replace: '6' -> '5',LD=1; replace: '6' -> '8',LD=1; replace: '6' -> '9',LD=1; replace: '7' -> '1',LD=1; replace: '7' -> '2',LD=1; replace: '7' -> '3',LD=1; replace: '7' -> '4',LD=1; replace: '7' -> '5',LD=1; replace: '7' -> '9',LD=1; replace: '8' -> '0',LD=1; replace: '8' -> '1',LD=1; replace: '8' -> '2',LD=1; replace: '8' -> '3',LD=1; replace: '8' -> '4',LD=1; replace: '8' -> '5',LD=1; replace: '8' -> '6',LD=1; replace: '8' -> '9',LD=1; replace: '9' -> '0',LD=1; replace: '9' -> '2',LD=1; replace: '9' -> '3',LD=1; replace: '9' -> '5',LD=1; replace: '9' -> '6',LD=1; replace: '9' -> '8',LD=2; delete: '0'; insert: '2',LD=2; delete: '1'; insert: '6',LD=2; delete: '1'; insert: '9',LD=2; delete: '1'; replace: '5' -> '06',LD=2; delete: '1'; replace: '8' -> '75',LD=2; delete: '1'; replace: '9' -> '5',LD=2; delete: '18'; insert: '33',LD=2; delete: '2'; insert: '1',LD=2; delete: '3'; insert: '6',LD=2; delete: '4'; insert: '3',LD=2; delete: '5'; insert: '0',LD=2; delete: '5'; insert: '3',LD=2; delete: '5'; insert: '9',LD=2; delete: '53',LD=2; delete: '55'; insert: '85',LD=2; delete: '58',LD=2; delete: '6'; insert: '5',LD=2; delete: '6'; insert: '6',LD=2; delete: '6'; replace: '6' -> '85',LD=2; delete: '7'; insert: '8',LD=2; delete: '7'; replace: '5' -> '99',LD=2; delete: '78',LD=2; delete: '8'; insert: '0',LD=2; delete: '8'; insert: '2',LD=2; delete: '8'; insert: '5',LD=2; delete: '8'; insert: '9',LD=2; delete: '80',LD=2; delet